In [216]:
csv_lang = "hi"

In [217]:
import pandas as pd
# inference_df = pd.read_csv(f"language_wise/{csv_lang}_intro.csv")
inference_df = pd.read_csv("user_study/hi_v2.csv")

In [218]:
# inference_df = inference_df.drop_duplicates(subset="human")

In [219]:
len(inference_df) - inference_df.isna().sum()

hi_intro                                 100
en_categories                            100
title                                    100
language                                 100
link                                     100
Age Group                                 97
Caste                                     30
Count                                     97
Highest Educational Qualification         31
Era                                       97
Ethnicity                                 97
Entity Existence Type                     97
Fame                                      97
Gender                                    97
Life Status                               97
Nationality                               97
Origin-of-Culture                         97
Religion                                  33
Role                                      34
Sex                                       34
Sexual Orientation                        34
Pronoun/Verb in Article Text             100
Pronoun/Ve

In [220]:
# inference_df = inference_df.drop(["en_categories"], axis=1)

In [221]:
# inference_df = inference_df.drop(["bpy", "bpy_intro"], axis=1)
inference_df = inference_df.drop(columns=["Unnamed: 0"], axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [222]:
len(inference_df) - inference_df.isna().sum()

hi_intro                                 100
en_categories                            100
title                                    100
language                                 100
link                                     100
Age Group                                 97
Caste                                     30
Count                                     97
Highest Educational Qualification         31
Era                                       97
Ethnicity                                 97
Entity Existence Type                     97
Fame                                      97
Gender                                    97
Life Status                               97
Nationality                               97
Origin-of-Culture                         97
Religion                                  33
Role                                      34
Sex                                       34
Sexual Orientation                        34
Pronoun/Verb in Article Text             100
Pronoun/Ve

In [223]:
# # non_null = inference_df.dropna()

# # print(len(non_null))
# inference_df = inference_df.iloc[:30]
# inference_df = inference_df[inference_df["title"].__eq__("Chandrayee Ghosh")]
# inference_df = inference_df[inference_df["human"].__eq__("http://www.wikidata.org/entity/Q890911")]
# # non_null = inference_df[~inference_df["bpy_intro"].isna()]

In [224]:
print(inference_df)

                                             hi_intro  \
0   ऋषि सुनक (जन्म 12 मई 1980) भारतीय मूल के ब्रिट...   
1   विद्या देवी भंडारी (नेपाली: विद्यादेवी भण्डारी...   
2   ग्रेटा थनबर्ग (ग्रेटा टिनटिन एलोनोरा एर्नमन थन...   
3   दुर्गा या आदिशक्ति हिन्दुओं की प्रमुख देवी मान...   
4   सरस्वती हिन्दू धर्म की प्रमुख वैदिक एवं पौराणि...   
..                                                ...   
95  मैडोना (मैडोना लुईस चिकोने जन्म, 16 अगस्त 1958...   
96  व्लदिमिर इल्यिच उल्यानोव, जिन्हें लेनिन के नाम...   
97  शकुनि या शकुनी गंधार साम्राज्य का राजा था। यह ...   
98  जटायु रामायण के एक प्रसिद्ध गरुड़ पात्र है। जब...   
99  दाउद इब्राहिम कासकर एक कुख्यात आतंकवादी है जो ...   

                                        en_categories                title  \
0   Category:1980 births|Category:21st-century pri...          Rishi Sunak   
1   Category:1961 births|Category:21st-century Nep...  Bidya Devi Bhandari   
2   Category:2003 births|Category:21st-century Swe...       Greta Thunberg   
3  

In [225]:
# import requests

# USER_AGENT = "WikiIntroExtractor/1.0 (Sourabarata Mukherjee; soura1990@gmail.com)"

# def get_introductory_text(wiki_url):
#     if not wiki_url or wiki_url is None:
#         return None
#     try:
#         # Extract the title from the URL
#         encoded_title = wiki_url.split("/")[-1]
#         title = requests.utils.unquote(encoded_title)
        
#         # API endpoint and parameters
#         api_url = f"https://{wiki_url.split('/')[2]}/w/api.php"
#         params = {
#             "action": "query",
#             "format": "json",
#             "prop": "extracts",
#             "explaintext": True,
#             "titles": title
#         }
#         headers = {"User-Agent": USER_AGENT}
        
#         # Make the API request
#         response = requests.get(api_url, headers=headers, params=params, timeout=60)
#         if response.status_code != 200:
#             print(response)
#             return None
#         data = response.json()
#         pages = data.get("query", {}).get("pages", {})
#         page = next(iter(pages.values()))
#         intro = page.get("extract", None)
        
#         if intro == "":
#             return None
        
#         return intro
#     except Exception as e:
#         return None

In [226]:
# output = get_introductory_text(non_null["hi"].iloc[0])
# print(output)

In [227]:
# with pd.option_context("display.max_rows", 100, "display.max_columns", 200) : 
#     print(non_null.isna())
#     print(non_null["hi_intro"].iloc[0])

In [228]:
import json
import re

def postprocess(input_data):
    # Extract content inside the first { } using regex
    match = re.search(r'\{[\s\S]*\}', input_data)  # Ensures capturing nested JSON structures
    
    if not match:
        raise ValueError("No valid JSON-like dictionary found in the input.")

    valid_json_str = match.group(0)

    # Step 1: Ensure proper JSON formatting
    valid_json_str = valid_json_str.replace("'", '"')  # Convert single quotes to double quotes

    # Step 2: Remove trailing commas (which are invalid in JSON)
    valid_json_str = re.sub(r',\s*([\]}])', r'\1', valid_json_str)

    try:
        # Step 3: Load as a Python dictionary
        data = json.loads(valid_json_str)
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", str(e))
        print("Problematic JSON String:", valid_json_str)  # Debugging output
        raise  # Rethrow exception after printing

    return data

In [229]:
# !pip install openai
# !pip install tiktoken
# !pip install anthropic

In [230]:
from openai import OpenAI
from tqdm import tqdm
import openai
import json
import os
from openai import AzureOpenAI
import os 
import tiktoken
import anthropic


# client = OpenAI()


# client = anthropic.Anthropic(
#     # defaults to os.environ.get("ANTHROPIC_API_KEY")
#     api_key=ANTHROPIC_KEY,
# )

# # print(message.content)

     
encoding = tiktoken.get_encoding("cl100k_base")
# encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# encoding = tiktoken.encoding_for_model("GPT-4o-mini Global Deployment")

## Fill credentials here
api_key = os.getenv("AZURE_OPENAI_KEY", "")
endpoint = os.getenv("ENDPOINT_URL", "")
# deployment = os.getenv("DEPLOYMENT_NAME", "gpt35turbo")

# Define deployment names for o1 and gpt-4o models
deployment_o1 = os.getenv("DEPLOYMENT_NAME_O1", "o1")
deployment_4o = os.getenv("DEPLOYMENT_NAME_4O", "gpt-4o")
deployment_4o_mini = os.getenv("DEPLOYMENT_NAME_4O_MINI", "gpt-4o-mini")
deployment_sonar = os.getenv("DEPLOYMENT_NAME_SONAR", "sonar-pro")
deployment_o1_mini = os.getenv("DEPLOYMENT_NAME_O1_MINI", "o1-mini")


# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-05-01-preview",
)

# client = OpenAI(api_key=api_key, base_url="https://api.perplexity.ai")

pronouns = """Examples of Honorific Pronouns: "वे", "उनका", "उनके ", "उनको", "उन्होंने", "उन्हें", "उनसे", "इनका", "इनकɃ", "इनके ", "इनको"
Examples of Non-Honorific Pronouns: "वह", "वो", "उसने", "उसको", "उसे", "उसका", "उसके ", "उसकɃ", "उनका", "उनके ", "उनकɃ", "उनको", "इसका", "इसकɃ", "इसके ", "इनका", "इनकɃ", "इनके ", "इनको", "उनसे", "उन्होंने", "उन्हें"
"""

verbs = """Examples of Honorific Verbs: "उन्होंने किया", "उन्होंने कहा", "करते हैं", "मिलीं", "थे"
Examples of Non-Honorific Verbs: "उसने किया", "उसने कहा", "करता है", "मिली", "था"
"""


# pronouns = """- Examples of Honorific Pronouns: "তিনি", "তাঁরা", "তাঁর", "তাঁদের", "ইনি", "উনি", "ওঁনারা", "যিনি"
#         - Examples of Non-Honorific Pronouns: "সে", "ও", "ওরা", "তারা", "তার", "ওর", "তাদের", "ওদের"
# """


# verbs = """- Examples of Honorific Verbs: "যান", "বলেন", "লেখেন", "করেছেন", "বলেছেন", "দেখেছেন", "শুনেছেন", "লিখেছেন", "করবেন", "লিখবেন", "করছেন"
#         - Examples of Non-Honorific Verbs: "যায়", "বলে", "লেখে", "করেছে", "বলেছে", "দেখেছে", "শুনেছে", "লিখেছে", "করবে", "লিখবে"
# """
# pronouns = ""
# verbs = ""

if pronouns != "" or verbs != "" : 
    examples = f"""Examples: 
        {pronouns}
        {verbs}
    """
    
else : 
    examples = ""

def generate_output(title, article_text, en_categories, lang_categories, language, model_type="o1"):
    # Choose the appropriate deployment name
    if "gpt-4o" in model_type.lower():
        # print("Using gpt-4o")
        deployment = deployment_4o
    elif model_type == "o1":
        deployment = deployment_o1  # Default to o1 model
        
    elif model_type == "gpt-4o-mini": 
        deployment = deployment_4o_mini
        
    elif model_type == "sonar-pro" : 
        deployment = deployment_sonar
        
    elif model_type == "o1-mini": 
        deployment = deployment_o1_mini
        
    else : 
        deployment = model_type
        
    messages=[
        {
        "role": "system",
        "content": "You will be provided with statements, and your task is to convert them to standard English."
        },
        {
        "role": "user",
        "content": f"""We need your help to specify the usage of third-person honorific and non-honorific pronouns, and/or verbs in the {language} article text titled {title}.
        
        {examples}
        
        Article Text:

        {article_text}
        
        Categories in English:

        {en_categories}
        
        
        Task:

        Please carefully review the article text, categories, and select the value for the following features for the article:

        - Age Group: 
            - Adult (18-60)
            - Juvenile (under 18)
            - Old (60+)
            - Not Applicable (for abstract entities like concepts or gods)

        - Caste (if applicable): 
            - Labor and Service Class: Includes workers, servants, and manual laborers (e.g., Shudras in Hinduism, serfs in feudal Europe).
            - Marginalized Groups: Includes historically excluded or oppressed groups (e.g., Dalits in India, Burakumin in Japan, untouchables in other societies).
            - Merchant and Artisan Class: Includes traders, craftsmen, and business people (e.g., Vaishyas in Hinduism, merchant guilds in medieval Europe, bazaar merchants in Islamic societies).
            - Religious and Priesthood Class: Includes clergy, priests, and spiritual leaders (e.g., Brahmins in Hinduism, clergy in Christianity, imams in Islam).
            - Warrior and Ruler Class: Includes nobility, royalty, and military elites (e.g., Kshatriyas in Hinduism, samurai in Japan, knights in medieval Europe).
            - Not Applicable (if none apply, e.g., abstract entities, non-human beings, deities, or supernatural beings)

        - Count: 
            - Plural (group or category)
            - Singular (individual entity)
            - Not Applicable (abstract concepts, mixed subjects)

        - Highest Educational Qualification: Each category represents the highest level of education an individual or entity has completed.
            - Doctoral: Holds a Doctorate (Ph.D. or equivalent) from an accredited institution.
            - Elementary: Has completed primary education, which typically covers early schooling years before secondary education.
            - Postgraduate: Completed a Master’s degree or equivalent postgraduate qualification.
            - Secondary: Completed secondary education, typically equivalent to high school.
            - Undergraduate: Holds a Bachelor’s degree or equivalent from a recognized higher education institution.
            - Not Applicable (if none apply, e.g., abstract entities, non-human beings, deities, or supernatural beings).

        - Era: 
            - Historical (before 1800)
            - Modern (after 1800)

        - Ethnicity: 
            - African: Includes Sub-Saharan Africans (e.g., Nigerians, Kenyans) and North Africans (e.g., Egyptians, Moroccans).
            - American: Includes Native Americans, First Nations (Canada), and Indigenous peoples of Central and South America, as well as those of European, African, Asian, and other ancestries who have historically formed distinct cultural identities within the region.
            - Asian: Includes East Asians (e.g., Chinese, Japanese, Koreans), South Asians (e.g., Indians, Pakistanis, Bangladeshis), and Southeast Asians (e.g., Indonesians, Vietnamese, Thais).
            - Australian: Indigenous peoples of Australia with distinct cultural and linguistic heritage.
            - European: Includes Western Europeans (e.g., French, Germans), Eastern Europeans (e.g., Russians, Poles), and Southern Europeans (e.g., Italians, Greeks).
            - Latino or Hispanic: Includes people from Latin America and the Caribbean, encompassing diverse ethnic backgrounds (e.g., Mexicans, Brazilians, Cubans).
            - Middle Eastern and Central Asian: Includes Arabs, Persians (Iranians), Turks, Kurds, and ethnicities from Central Asia (e.g., Uzbeks, Kazakhs).
            - Mixed or Multiracial: Includes individuals with ancestry from multiple ethnic backgrounds.
            - Not Applicable (if none of the above apply, e.g., abstract entities)
            - Unknown (if none of the above apply)

        - Entity Existence Type: 
            - Animal & Fictional character
            - Animal & Mythological character
            - Animal & Real
            - God
            - Human & Fictional character
            - Human & Mythological character
            - Human & Real
            - Other Being

        - Fame (Sentiment associated with the entity): 
            - Controversial
            - Famous
            - Infamous

        - Gender: 
            - Boy/Man
            - Girl/Woman
            - Gender Neutral
            - Non-Binary
            - Transgender
            - Not Applicable (if none apply, e.g., abstract entities)

        - Life Status: 
            - Alive
            - Dead
            - Not Applicable (for abstract entities like concepts or gods)

        - Nationality: The native country of the entity (e.g., India, Bangladesh, France) if applicable, else Unknown.

        - Origin-of-Culture: 
            - Exotic (from a different demography as compared to the speakers of {language} language)
            - Native (same demography as the speakers of {language} language)

        - Pronoun/Verb in Article Text: Identify the type of pronouns/verbs used in the article text with respect to {title} and select one of the following option:
            - Honorific: Select this option if most of the pronouns and/or verbs used in the article are formal, respectful, or honorific.
            - Non-Honorific: Select this option if most of the pronouns and/or verbs used in the article are informal, casual, or non-honorific.
            - Not Applicable: Select this option if the article does not contain any pronouns or verbs at all that can help classify it under either of the above two categories(honorific or non-honorific).

            If the article has mixed usage of honorific and non-honorific pronouns/verbs, list down all the pronouns/verbs used for {title} and classify the ones having the majority.  

        - Religion: 
            - Atheism
            - Buddhism
            - Christianity
            - Hinduism
            - Indigenous or Tribal Religions (e.g., Native American spirituality, African traditional religions, Aboriginal beliefs)
            - Islam
            - Jainism
            - Judaism
            - Sikhism
            - Not Applicable (if none apply, e.g., abstract entities)

        - Role: Choose the best fitting category based on the following (use your judgment if the entity fits multiple categories):
            - Activists and Reformers (e.g., civil rights, environmental, social activists)
            - Arts and Culture
            - Business and Economy
            - Criminals (e.g., criminal, terrorist, outlaw, fraudster)
            - Deity
            - Education and Academia
            - Entertainment (use this over Arts and Culture if the entity is primarily involved in entertainment)
            - Infamous/Controversial Activists (e.g., controversial or notorious figures in activism or leadership)
            - Law and Justice
            - Literature and Philosophy
            - Media and Journalism
            - Medicine and Healthcare
            - Military
            - Politics and Governance
            - Public Service
            - Religion and Spirituality
            - Royalty and Nobility
            - Science and Technology
            - Sports
            - Not Applicable (if none of the above apply)

        - Sex: 
            - Female
            - Male
            - Not Applicable (if none apply, e.g., abstract entities or non-living beings)

        - Sexual Orientation: 
            - Bisexual
            - Heterosexual
            - Homosexual
            - Not Applicable (if none apply, e.g., abstract entities)

        Output Format:

        ONLY provide your answers in dictionary format, where the keys are the feature names, and the values correspond only from the options provided above. Do NOT include any additional text besides the dictionary. Example format:

        {{
        'Age Group': 'Adult',
        'Caste': 'Marginalized Groups',
        'Count': 'Singular',
        'Highest Educational Qualification': 'Doctoral',
        'Entity Existence Type': 'Human',
        'Era': 'Modern',
        'Ethnicity': 'Asian',
        'Fame': 'Famous',
        'Gender': 'Boy/Man',
        'Life Status': 'Dead',
        'Nationality': 'India',
        'Origin-of-Culture': 'Native',
        'Pronoun/Verb in Article Text': 'Honorific',
        'Religion': 'Hinduism',
        'Role': 'Politics and Governance',
        'Sex': 'Male',
        'Sexual Orientation': 'Heterosexual',
        }}
        """
        }
        ]
    
    # print(messages[1]["content"])
    
    response = client.chat.completions.create(
      model=deployment,
      messages = messages,
      temperature=0,
      max_tokens=256,
      top_p=1
    ) 
    
    # print(messages[1]["content"])
    
    # response = client.messages.create(
    # model="claude-3-5-sonnet-20241022",
    # max_tokens=1024,
    # messages=messages,
    # )
    
    out = response
    
    return out, messages[1]["content"]

In [231]:
import requests
import urllib

def get_wikipedia_categories(page_title, language):
    # Decode back to original representation
    page_title = urllib.parse.unquote(page_title)

    # print(decoded_text)
    url = f"https://{language}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
            "prop": "categories",
            "exintro": True,
            "explaintext": True,
            "cllimit": "max",  # Fetch all categories
            "redirects": 1,    # Automatically resolve redirects
            "titles": page_title,
            "format": "json",
            "clshow": "hidden",
    }
    print(url)
    response = requests.get(url, params=params, timeout=60)
    if response.status_code != 200:
        print("Bad request")
        print(response.status_code)
        return []

    categories = []

    data = response.json()
    # print(data)
    pages = data["query"]["pages"]
    print(pages)
    for key in pages.keys():
        if "categories" in pages[key]:
            # print(pages[key]["categories"][0])
            categories.extend([cat["title"] for cat in pages[key]["categories"]])
            # print(categories)
    return categories



In [234]:
import json
import os
from tqdm import tqdm
import urllib.parse

# URL-encoded string

# Pricing details
model_type = "GPT-4o-2024-1120 Global"
input_price_per_token = 0.0025 / 1000
output_price_per_token = 0.001 / 1000
# output_file = f"{csv_lang}.json"

#### Tester
output_file = "test_hi.json"

# Ensure the output file exists or create an empty JSON file
if not os.path.exists(output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump([], f)  # Initialize an empty JSON array

# Load existing JSON file if available
try:
    with open(output_file, "r", encoding="utf-8") as f:
        existing_data = json.load(f)
except (json.JSONDecodeError, FileNotFoundError):
    existing_data = []  # If the file is corrupt or missing, start fresh
    
existing_titles = {entry["title"] for entry in existing_data}
print(len(existing_titles))
# Convert existing data into a dictionary for quick lookup (title + language as key)
existing_entries = {(entry["title"], entry["language"]) for entry in existing_data}

# Initialize total price counter
total_price = 0

lang_id_mapper = {
    "hi" : "Hindi",
    "ko" : "Korean",
    "ja" : "Japanese",
    "bn" : "Bengali",
    "mr" : "Marathi",
    "en" : "English",
    "th" : "Thai",
    "jv" : "Javanese",
    "ne" : "Nepali", 
    "ta" : "Tamil",
    "ur" : "Urdu",
    "bpy" : "Bishnupriya Manipuri"
}

def calculate_price(prompt_tokens, completion_tokens):
    """Calculate the price based on token usage."""
    return (prompt_tokens * input_price_per_token) + (completion_tokens * output_price_per_token)

for i, row in tqdm(inference_df.iterrows()):
    en_category = row["en_categories"]
    # en_category = "|".join(en_categories)
    # print(en_category)
    col = "hi_intro"
    # for col in inference_df.columns:
        
        # if "intro" in col and "en" not in col and csv_lang in col: 
    # if pd.isna(row[col]) or row[col] == "" or not row[col] : 
    #     continue
        
    title_language_pair = (row["title"], col.split("_")[0])

    # Skip if the data already exists
    if title_language_pair in existing_entries:
        # print(f"[INFO] Skipping {title_language_pair} (Already Exists)")
        continue
    
    lang_str = col.split("_")[0]
    # lang_category = row[f"{lang_str}_categories"]
    # lang_category = "|".join(lang_categories)
    # print(lang_category)
    
    
    

    # Generate API response
    outcome, prompt = generate_output(row["title"], row[col], en_category, "", lang_id_mapper[lang_str], model_type)

    # Extract token usage
    prompt_tokens = outcome.usage.prompt_tokens
    completion_tokens = outcome.usage.completion_tokens

    # Calculate price
    single_price = calculate_price(prompt_tokens, completion_tokens)
    total_price += single_price

    # Prepare JSON output
    json_out = postprocess(outcome.choices[0].message.content)
    json_out["title"] = row["title"]
    json_out["article_text"] = row[col]
    json_out["categories"] = en_category
    json_out["language_categories"] = ""
    json_out["language"] = lang_str
    json_out["price"] = single_price
    json_out["prompt"] = prompt

    # Append new result to file immediately
    with open(output_file, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)  # Load existing data
        except json.JSONDecodeError:
            data = []  # If file is empty/corrupt, start fresh

        data.append(json_out)  # Append new data
        
        f.seek(0)  # Move cursor to start of file before writing
        json.dump(data, f, indent=4, ensure_ascii=False)
        f.truncate()  # Remove extra trailing content if any

    # Mark as processed
    existing_entries.add(title_language_pair)
    # print(f"[INFO] Saved: {title_language_pair}")
            

print("Total Price for {} prompts: ${:.4f}".format(len(existing_entries), total_price))

50


100it [02:14,  1.35s/it]

Total Price for 100 prompts: $0.3738


In [241]:
with open(output_file, "r+", encoding="utf-8") as f:
        try:
            data = json.load(f)  # Load existing data
        except json.JSONDecodeError:
            data = []  # If file is empty/corrupt, start fresh

In [242]:
annotated_data = pd.DataFrame(data)

In [243]:
annotated_data.head()

,Age Group,Caste,Count,Highest Educational Qualification,Entity Existence Type,Era,Ethnicity,Fame,Gender,Life Status,...,Role,Sex,Sexual Orientation,title,article_text,categories,language_categories,language,price,prompt
0,Adult,Merchant and Artisan Class,Singular,Postgraduate,Human & Real,Modern,Asian,Famous,Boy/Man,Alive,...,Politics and Governance,Male,Heterosexual,Rishi Sunak,ऋषि सुनक (जन्म 12 मई 1980) भारतीय मूल के ब्रिट...,Category:1980 births|Category:21st-century pri...,,hi,0.007096,We need your help to specify the usage of thir...
1,Adult,Not Applicable,Singular,Not Applicable,Human & Real,Modern,Asian,Famous,Girl/Woman,Alive,...,Politics and Governance,Female,Not Applicable,Bidya Devi Bhandari,विद्या देवी भंडारी (नेपाली: विद्यादेवी भण्डारी...,Category:1961 births|Category:21st-century Nep...,,hi,0.006175,We need your help to specify the usage of thir...
2,Juvenile,Not Applicable,Singular,Not Applicable,Human & Real,Modern,European,Famous,Girl/Woman,Alive,...,Activists and Reformers,Female,Not Applicable,Greta Thunberg,ग्रेटा थनबर्ग (ग्रेटा टिनटिन एलोनोरा एर्नमन थन...,Category:2003 births|Category:21st-century Swe...,,hi,0.006941,We need your help to specify the usage of thir...
3,Not Applicable,Not Applicable,Singular,Not Applicable,God,Historical,Not Applicable,Famous,Girl/Woman,Not Applicable,...,Deity,Female,Not Applicable,Durga,दुर्गा या आदिशक्ति हिन्दुओं की प्रमुख देवी मान...,Category:All articles with unsourced statement...,,hi,0.008840,We need your help to specify the usage of thir...
4,Not Applicable,Not Applicable,Singular,Not Applicable,God,Historical,Not Applicable,Famous,Girl/Woman,Not Applicable,...,Deity,Female,Not Applicable,Saraswati,सरस्वती हिन्दू धर्म की प्रमुख वैदिक एवं पौराणि...,Category:All articles covered by WikiProject W...,,hi,0.009890,We need your help to specify the usage of thir...


In [244]:
annotated_data.iloc[2]["article_text"]

'ग्रेटा थनबर्ग (ग्रेटा टिनटिन एलोनोरा एर्नमन थनबर्ग) (जन्म 3 जनवरी 2003) स्वीडन की एक पर्यावरण कार्यकर्ता हैं जिनके पर्यावरण आंदोलन/आन्दोलन को अंतरराष्ट्रीय/अन्तरराष्ट्रीय ख्याति मिली है। स्वीडन की इस किशोरी के आंदोलनों/आन्दोलनों के फलस्वरूप विश्व के नेता अब जलवायु परिवर्तन पर कार्य करने के लिए विवश हुए हैं। अगस्त 2018 में, 15 की उम्र में, थनबर्ग ने स्कूल से समय निकालकर हाथ में स्वीडन की भाषा में "Skolstrejk för klimatet " ( जलवायु के लिए स्कूलबन्दी (स्कूलबंदी)) लिखी तख्ती लिए स्वीडन की संसद के बाहर प्रदर्शन करना शुरू किया। 11 दिसम्बर 2019 को इन्हे \'टाइम पर्सन ऑफ़ द ईयर\' पुरस्कार प्रदान किया गया। ग्रेटा अपने सीधे-साधे शब्दों में बात करने के लिए भी जानी जातीं हैं। अपनी सार्वजनिक सभाओं में और राजनीतिक नेताओं के साथ वार्ता में वे जलवायु संकट पर तुरन्त (तुरंत) कार्वाई करने का आग्रह करतीं हैं।'

In [245]:
annotated_data.iloc[2]

Age Group                                                                     Juvenile
Caste                                                                   Not Applicable
Count                                                                         Singular
Highest Educational Qualification                                       Not Applicable
Entity Existence Type                                                     Human & Real
Era                                                                             Modern
Ethnicity                                                                     European
Fame                                                                            Famous
Gender                                                                      Girl/Woman
Life Status                                                                      Alive
Nationality                                                                     Sweden
Origin-of-Culture                          

In [246]:
annotated_data.to_csv("llm_annotation_v2/hi_10.csv")

In [198]:
annotated_data = pd.read_csv("hi_annotated.csv")

In [83]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def merge_columns(row) : 
    col1 = "Pronoun/Verb in Article Text"
    col2 = "Pronoun/Verb in Article text"
    
    if row[col1] == None or pd.isna(row[col1]) : 
        return row[col2]
    
    if row[col2] == None or pd.isna(row[col2]) : 
        return row[col1]
    
    return row[col1] + row[col2]

# # Load JSON data from file
output_file = "output.json"

with open(output_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract relevant data into a DataFrame
df = pd.DataFrame(data)

df = annotated_data

df["honorific"] = df.apply(merge_columns, axis=1)

final_df = df.drop(["Pronoun/Verb in Article text"], axis = 1)

final_df.to_csv(f"Wiki_annotations_{csv_lang}.csv")

# Ensure required columns exist
if "language" not in df.columns: 
    raise ValueError("Missing required columns in JSON file.")

print(df.info())

# Convert 'honorific' field to boolean (assuming values like 'yes'/'no' or True/False)
df["honorific"] = df["honorific"].astype(str).str.lower().map({"Honorific": 1, "honorific": 1, "non-honorific": 0, "Non-Honorific": 0})

# Aggregate counts per language
summary = df.groupby("language")["honorific"].agg(["sum", "count"])
summary.rename(columns={"sum": "honorific_count", "count": "total_count"}, inplace=True)

# Calculate non-honorific count and percentages
summary["non_honorific_count"] = summary["total_count"] - summary["honorific_count"]
summary["honorific_percentage"] = (summary["honorific_count"] / summary["total_count"]) * 100
summary["non_honorific_percentage"] = 100 - summary["honorific_percentage"]

# Sorting by honorific percentage for better visualization
summary = summary.sort_values(by="honorific_percentage", ascending=False)

# Plot
languages = summary.index
honorific_counts = summary["honorific_count"]
non_honorific_counts = summary["non_honorific_count"]
honorific_percentages = summary["honorific_percentage"]

x = np.arange(len(languages))  # X-axis positions

fig, ax = plt.subplots(figsize=(12, 6))

# Stacked bar plot
bars1 = ax.bar(x, non_honorific_counts, label="Non-Honorific", color="lightblue")
bars2 = ax.bar(x, honorific_counts, bottom=non_honorific_counts, label="Honorific", color="orange")

# Labels and Titles
ax.set_xlabel("Language")
ax.set_ylabel("Count")
ax.set_title("Honorific vs. Non-Honorific Mentions by Language")
ax.set_xticks(x)
ax.set_xticklabels(languages, rotation=45, ha="right")
ax.legend()

# Adding percentage labels on bars
for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
    height1 = bar1.get_height()
    height2 = bar2.get_height()
    total_height = height1 + height2

    # Show honorific percentage on top
    ax.text(bar1.get_x() + bar1.get_width() / 2, total_height + 1, f"{honorific_percentages.iloc[i]:.1f}%", 
            ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

KeyError: 'Pronoun/Verb in Article Text'

In [ ]:
df["language"].value_counts()

In [57]:
# Pivot the table to get one column per language
df_pivot = df.pivot_table(index="title", columns="language", values="honorific", aggfunc="max", fill_value=0)

# Reset index to turn title back into a column
df_pivot.reset_index(inplace=True)

df_pivot.to_csv("pivot.csv")

In [ ]:
df.info()

In [59]:
from sklearn.preprocessing import LabelEncoder

df.drop(["Pronoun/Verb in Wiki Article"], axis=1)

lr_df = df.copy()
lr_df = lr_df.drop(["article_text", "categories", "title", "Entity"], axis=1)
lr_df = lr_df[lr_df["language"].str.contains("ja")].reset_index()

encoders = {}

for col in lr_df.columns :     
    col_encoder = LabelEncoder()
    lr_df[col] = col_encoder.fit_transform(lr_df[col])
    encoders[col] = col_encoder

In [ ]:
column = "Era"

mapping = {}

inverse = encoders[column].inverse_transform(lr_df["Era"])

for i, row in lr_df.iterrows() : 
    val = row[column]
    
    if val not in mapping : 
        mapping[val] = inverse[i]
        
print(mapping)

In [ ]:
lr_df.head(50)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(lr_df.drop(["language", "honorific", "Pronoun/Verb in Wiki Article"], axis=1), lr_df["honorific"])

lr = LinearRegression()

lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

print(lr.coef_)

print(sum((y_pred-y_test)2))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

x_train, x_test, y_train, y_test = train_test_split(lr_df.drop(["language", "honorific", "Pronoun/Verb in Wiki Article"], axis=1), lr_df["honorific"])

lr = LogisticRegression()

lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

print(lr.coef_)

print(classification_report(y_pred=y_pred, y_true=y_test))
print(lr_df.columns)